In [127]:
import math
import numpy as np
import pandas as pd
import datetime as dt
import scipy.stats as stats
import matplotlib.pyplot as plt
from pandas_datareader import  data as pdr
import yfinance as yf


In [128]:
# Option parameters
K = 98.01           # Strike price(where we can either buy or sell incaase of a call option
r = 0.015           # Risk-free rate (%)(return on an investment with  zero risk of financial loss


In [129]:

# Fetch historical price data from Yahoo Finance
ticker = 'AAPL'
start_date = '2022-01-17'  # Start date which wil act as a reference date
end_date = '2022-03-17'    # End date is the expiration date of the option
data = yf.download(ticker, start=start_date, end=end_date)



[*********************100%%**********************]  1 of 1 completed


In [130]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-18,171.509995,172.539993,169.410004,169.800003,167.667892,90956700
2022-01-19,170.000000,171.080002,165.940002,166.229996,164.142700,94815000
2022-01-20,166.979996,169.679993,164.179993,164.509995,162.444275,91420500
2022-01-21,164.419998,166.330002,162.300003,162.410004,160.370667,122848900
2022-01-24,160.020004,162.300003,154.699997,161.619995,159.590591,162294600
2022-01-25,158.979996,162.759995,157.020004,159.779999,157.773712,115798400
2022-01-26,163.500000,164.389999,157.820007,159.690002,157.684814,108275300
2022-01-27,162.449997,163.839996,158.279999,159.220001,157.220718,121954600
2022-01-28,165.710007,170.350006,162.800003,170.330002,168.191208,179935700


In [131]:
# Use the last available adjusted closing price as the stock price
S = data['Adj Close'].iloc[-1] 
S

157.78684997558594

In [132]:

# Calculate time to expiration (T)
expiration_date = dt.datetime.strptime(end_date, '%Y-%m-%d').date()
reference_date = dt.datetime.strptime(start_date, '%Y-%m-%d').date()
T = (expiration_date - reference_date).days / 365  # Time to expiration (in years)


In [133]:
T

0.16164383561643836

In [134]:
# Volatility calculation
vol = np.std(data['Adj Close'].pct_change()) * np.sqrt(252)  # Annualized historical volatility (%)


In [135]:
vol

0.32883165723629204

In [143]:
# Monte Carlo simulation parameters

N = 10  # Number of time steps
M = 1000  # Number of simulations


In [144]:
# Define function to calculate delta of an option
def delta_calc(S, K, T, r, sigma, option_type='c'):
     # Calculate delta of an option using the Black-Scholes formula.
    
    # d1 => represents the standardized measure of how many standard deviations the current stock price is from the strike price
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    if option_type == 'c':
        return stats.norm.cdf(d1, 0, 1)
    elif option_type == 'p':
        return -stats.norm.cdf(-d1, 0, 1)
    else:
        raise ValueError("Invalid option type. Use 'c' for Call or 'p' for Put.")


In [145]:
# Define function to calculate Black-Scholes call option price
def black_scholes_call_price(S, K, T, r, sigma):
    # Calculate the price of a European call option using the Black-Scholes formula.
    
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    # d2 =>  represents a standardized measure of how many sd the current stock price is from the strike price when adjusted for time and volatility.
    d2 = d1 - sigma * np.sqrt(T)
    return S * stats.norm.cdf(d1) - K * np.exp(-r * T) * stats.norm.cdf(d2)


In [146]:
#  constants for Monte Carlo simulation
dt = T / N   
nudt = (r - 0.5 * vol**2) * dt
volsdt = vol * np.sqrt(dt)
erdt = np.exp(r * dt)


In [147]:
# Monte Carlo simulation
Z = np.random.normal(size=(N, M))
delta_St = nudt + volsdt * Z  #am geting the changes in the asset price at each time step.
ST = S * np.cumprod(np.exp(delta_St), axis=0)

In [148]:
ST

array([[150.43740894, 167.97927632, 158.54399016, ..., 167.44821307,
        163.87822867, 156.95842469],
       [144.70361873, 166.25016022, 164.83029849, ..., 178.21799288,
        162.11773218, 152.06915716],
       [143.59940865, 178.0599484 , 158.68103289, ..., 167.53783355,
        173.50220001, 144.18614893],
       ...,
       [151.23601881, 152.11729056, 149.67611474, ..., 139.6243192 ,
        171.56717064, 124.16521025],
       [158.33225488, 157.30236394, 164.46323607, ..., 132.0604012 ,
        179.1871636 , 114.92429953],
       [161.2079974 , 151.95925373, 167.80799716, ..., 132.64314008,
        180.23883999, 108.31765368]])

### Note

1. The array ST represents the asset price trajectories over time .
2. Each row of the array corresponds to a specific time step whereas   each column represents a different simulation or path.

In [149]:
# Calculate differences in asset prices and refining the shape to be good

price_differences = (ST[1:] - ST[:-1] * erdt)
price_differences = np.vstack([np.zeros((1, M)), price_differences])

# Correcting the shape mismatch
cv = np.cumsum(deltaSt * price_differences, axis=0)


### Note

CV contains the cumulative values of the option's payoff changes over time for each simulation

In [150]:
price_differences.ndim

2

In [151]:
price_differences

array([[  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [ -5.77027055,  -1.76985026,   6.24786218, ...,  10.72917443,
         -1.80023616,  -4.92732918],
       [ -1.13930001,  11.76947332,  -6.18923615, ..., -10.72337633,
         11.34515507,  -7.91988426],
       ...,
       [  5.66610666,  -3.13776776,  -0.82430994, ..., -10.57556285,
          2.24125601,   7.7730243 ],
       [  7.05956207,   5.14818568,  14.7508256 , ...,  -7.59777622,
          7.57838876,  -9.27102018],
       [  2.83734772,  -5.38125527,   3.30487956, ...,   0.55071487,
          1.00822437,  -6.63451444]])

In [152]:
cv.ndim

2

In [153]:
print(cv)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-5.77001402e+00 -1.76950735e+00  6.24727709e+00 ...  1.07259047e+01
  -1.80022650e+00 -4.92708359e+00]
 [-6.90927297e+00  9.99986530e+00  5.83687448e-02 ...  5.58015921e-03
   9.54491091e+00 -1.28467581e+01]
 ...
 [ 5.52794543e-01 -1.61368536e+01 -9.13186847e+00 ... -2.81148128e+01
   7.39824649e+00 -3.30149030e+01]
 [ 7.61235661e+00 -1.09886679e+01  5.61895713e+00 ... -3.57125890e+01
   1.49766352e+01 -4.22859231e+01]
 [ 1.04497043e+01 -1.63699232e+01  8.92383668e+00 ... -3.51618741e+01
   1.59848596e+01 -4.89204376e+01]]


In [154]:
# Option pricing
CT = np.maximum(0, ST[-1] - K)  #calculates the payoff of the call option.
C0 = np.exp(-r * T) * np.mean(CT)
SE = np.std(np.exp(-r * T) * CT) / np.sqrt(M)

In [155]:
# Output results
print("Stock price (S): ${:.2f}".format(S))
print("Strike price (K): ${:.2f}".format(K))
print("Volatility (vol): {:.4f}".format(vol))
print("Risk-free rate (r): {:.4f}".format(r))
print("Time to expiration (T): {:.4f} years".format(T))

print("-------------------------------------")
print("Call option price: ${:.2f} +/- {:.4f}".format(C0, SE))

Stock price (S): $157.79
Strike price (K): $98.01
Volatility (vol): 0.3288
Risk-free rate (r): 0.0150
Time to expiration (T): 0.1616 years
-------------------------------------
Call option price: $59.35 +/- 0.6649


In [ ]:
# Stock price (S): $157.79
# Strike price (K): $98.01
# Volatility (vol): 0.3288
# Risk-free rate (r): 0.0150
# Time to expiration (T): 0.1616 years
# -------------------------------------
# Call option price: $59.97 +/- 0.6530